In [1]:
from baggingPU import BaggingClassifierPU
import numpy as np
import pandas as pd
from sklearn.svm import SVC
random_state=100

In [2]:
Features = pd.read_csv("../data/Microarray_PCA_df.csv",index_col=0)

In [3]:
Positive_sample = list(pd.read_csv("../Data/ARGC_MSIH_TILsH.txt",index_col=0).index)
EBV_sample = list(pd.read_csv("../Data/GSE62254_EBV_18.txt",index_col=0).index)
MSS_sample = list(pd.read_csv("../Data/GSE62254_MSS_226.txt",index_col=0).index)
Unlabeled_sample = EBV_sample+MSS_sample

In [6]:
data = Features.loc[Positive_sample+Unlabeled_sample,:]
grouping = [1]*len(Positive_sample) + [0]*len(Unlabeled_sample)

In [7]:
X = np.asmatrix(data)
y = np.asarray(grouping)

In [8]:
bc = BaggingClassifierPU(
    SVC(), 
    n_estimators = 1000,  # 1000 trees as usual
    max_samples = sum(y), # Balance the positives and unlabeled in each bag
    n_jobs = -1,
    random_state=random_state# Use all cores
)

In [9]:
bc.fit(X, y)
res = bc.oob_decision_function_[:,1]

/home/jovi/anaconda3/envs/r4/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [10]:
results = pd.DataFrame(index=Positive_sample+Unlabeled_sample)
results["PUL_Score"] = res

In [12]:
results.to_csv("../data/ACRG_PUL_score.csv")